In [5]:
from reader import ebook
from main import read_list,read_json
from typing import Optional
import requests
from dotenv import load_dotenv
import json
import os
os.environ['CURL_CA_BUNDLE'] = ''

In [6]:
load_dotenv()
API = os.getenv("HF_API")

headers = {
    "Authorization": f"Bearer {API}",
    "Content-Type": "application/json",
}
url = "https://api-inference.huggingface.co/models/mistralai/Mistral-7B-Instruct-v0.3/v1/chat/completions"


In [3]:
role='''(NOTE: Only output in JSON. Ensure the JSON format is valid, well-formed, and easy to parse.)
Input:  
1.Chapter Text: The current chapter to be analyzed.
2.Character List: A list of characters with their descriptions till now (This chapter).
3.Places list: list of places and their description till now (This chapter).
3.Previous Chapters' Summary: Context from earlier chapters.

Rules:  
1.Narrative Summary: Summarize the chapter, integrating context and key developments from previous chapters and create a self containing summary. end with to be continued.
2.Character List: add new characters to the list based on this chapter and Update existing character descriptions. If no characters are mentioned, return the same list as given.
3.Places: Include an updated description of any significant locations mentioned in this chapter, focusing on environment, weather, vibe, and structure.
4.Output Format: Ensure the output is valid and well-structured JSON.

Output:  
Generate a JSON object in this format:
{
  "summary": "Summary of the current chapter in context of previous chapters.",
  "characters": {
      "Character Name": "Updated or new description (age, looks, clothes, hair, body language) based on this chapter."
    },
  "places": {
      "Place Name": "Updated or new description (environment, weather, vibe, structure, etc.) based on this chapter."
  }
}

example:
{
  "summary": "In this chapter, Mira embarks on a courageous journey to seek the sun goddess Solara, facing numerous challenges along the way. As she navigates through the enchanted forest, her determination shines through, reflecting the hope of her village. The chapter highlights her growth and resilience against the encroaching darkness.",
  "characters": {
    "Mira": "A 16-year-old girl with long, wavy black hair and bright green eyes. She wears a simple tunic made of earth-toned fabric, suitable for travel, and has a determined expression. Mira carries a small satchel with essential supplies and exhibits adventurous body language, often standing tall with her shoulders back."
  },
  "places": {
    "Enchanted Forest": "A mystical area filled with vibrant flora and magical creatures, bathed in dappled sunlight. The air is rich with the sounds of nature, creating an atmosphere of wonder and adventure."
  }
}

'''


In [4]:
def msg(text: str, context: str = "Its first chapter", characters: dict = {}, places: dict = {}) -> list:
    js=json.dumps({
                "past_context": context,
                "text": text,
                "character_list": characters,
                "places_list": places})
    message = [
        {
            "role": "system",
            "content": role
        },
        {
            "role": "user",
            "content": json.dumps({
                "past_context": context,
                "text": text,
                "character_list": characters,
                "places_list": places
            }),
        },
    ]
    return message

In [3]:
def get_scene(messages) -> Optional[str]:
    data = {
        "messages": messages,
        "temperature": 0.7,
        "stream": False,
        "max_tokens":10000
    }
    response = requests.post(url, headers=headers, json=data)

    if response.status_code == 200:
        response_data = response.json()
        assistant_message = response_data["choices"][0]["message"]["content"]
        return assistant_message
    else:
        print(f"Error: {response.status_code}, {response.text}")
        return None

In [6]:
book=ebook("./books/HP.epub")
chapters = book.get_chapters()
_,text=zip(*chapters[:3]) 



In [7]:

output_dict={
    0:
        {
            "summary":"This is the first chapter",
            "characters":{},
            "places": {}
        }
            }

In [8]:
for idx,i in enumerate(text):
    mes=msg(text,output_dict[idx]["summary"])
    summary_characters=get_scene(mes)
    output_json=read_json(summary_characters)
    output_dict[idx+1]=output_json
    print(f"chapter: {idx} done")
    

ProxyError: HTTPSConnectionPool(host='api-inference.huggingface.co', port=443): Max retries exceeded with url: /models/mistralai/Mistral-7B-Instruct-v0.3/v1/chat/completions (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7bf2b9472110>: Failed to establish a new connection: [Errno 111] Connection refused')))

In [7]:
import requests

model_name = "mistralai/Mistral-7B-Instruct-v0.3"
url = f"https://huggingface.co/api/models/{model_name}"

response = requests.get(url,verify=False)
if response.status_code == 200:
    print(f"Model '{model_name}' is available.")
else:
    print(f"Model '{model_name}' is NOT available.")


ProxyError: HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /api/models/mistralai/Mistral-7B-Instruct-v0.3 (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x785d102e5cd0>: Failed to establish a new connection: [Errno 111] Connection refused')))

In [ ]:
output_dict

In [18]:
for num, dic in output_dict.items():
    if num==0:
        continue
    print(f"CHAPTER {num} :\n")
    
    sum,char,places=dic.items()
    print("Characters:")
    if len(char)>0 :
        for name,i in char[1].items():
            print(f"{name}: {i}")
    
    print("\nPlaces:")
    if len(places)>0:
        for name,i in places[1].items():
            print(f"{name}: {i}")
    print(f"\nSummary:\n{sum[1]}")
    print("\n-------------------------------------------------------------------------------------------------------------------")

CHAPTER 1 :

Characters:
Meursault: A young man in his 20s, working in Algiers. He is described as indifferent, calm, and somewhat detached from his emotions.

Places:
Algiers: A city in northern Africa where Meursault lives and works. It is described as hot and bustling.
Marengo: A town where the Home for Aged Persons is located, about fifty miles from Algiers. It is described as small and quiet.

Summary:
In this chapter, the protagonist, Meursault, receives the news of his mother's death and embarks on a journey to her funeral. He recalls their strained relationship and his indifference towards her death, causing discomfort among his colleagues. He also reflects on his past and his feelings towards life and death.

-------------------------------------------------------------------------------------------------------------------
CHAPTER 2 :

Characters:

Places:

Summary:
In this chapter, the protagonist, Meursault, receives the news of his mother's death and embarks on a journey to

In [406]:
b=read_json(a)

In [407]:
b

{'narrative_summary': 'The narrator recounts his past experiences, including the loss of his artistic dreams due to the lack of understanding from adults, and his encounter with a mysterious voice in the Sahara Desert that asked him to draw a sheep. This encounter rekindled his connection to childhood innocence.',
 'characters': {'The Narrator': 'An unnamed individual who is a seasoned traveler and pilot. He has a cynical view of adults and their lack of imagination. He carries a drawing of a mysterious little man he encountered in the Sahara Desert.'}}

In [402]:
char=b['characters']
summary=b['narrative_summary']

In [404]:
print("characters:")
for name,disc in char.items():
    print(f"{name}:\n{disc}")
print(f"\nSummary:\n{summary}")


characters:
The Narrator:
An unnamed character who is a grown-up, a pilot, and an artist at heart. He has a pessimistic view of adults and their lack of imagination. He wears a pilot's uniform and has a serious expression.

Summary:
The narrator reflects on his past, his lost artistic dreams, and his encounter with a mysterious voice in the Sahara Desert. This voice, later identified as the little prince, asks him to draw a sheep, reigniting his connection to childhood innocence.
